## Scraping Multiple Pages - San Francisco

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/CA/San_Francisco/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,214 Putnam St,3 Beds,4 Baths,"2,255 sqft",1996,Attached Garage,No,"$998,000"
1,1125 Vicente St,3 Beds,2 Baths,"2,018 sqft",1936,No Info,No,"$998,000"
2,646 De Haro St,2 Beds,2 Baths,"1,515 sqft",1934,Attached Garage,No,"$1,395,000"
3,89 Vienna St,2 Beds,1 Bath,,1900,Attached Garage,No,"$899,000"
4,134-136 Hartford St,2 Beds,2 Baths,"2,000 sqft",1900,0 Open Spaces,,"$895,000"
...,...,...,...,...,...,...,...,...
995,4 Jennifer Pl,1 Bed,1 Bath,"1,284 sqft (on 0.39 acres)",2000,Attached Garage,,"$998,000"
996,Residence #215 Plan in Fulton 555 by Z&L Prope...,1 Bed,1 Bath,708 sqft,2021,No Info,,"$719,000+"
997,2177 3rd St #505,1 Bed,1 Bath,719 sqft,2020,No Info,No,"$985,000"
998,85 Columbia Square St,6 Beds,5 Baths,"4,998 sqft",1921,Garage,,"$3,500,000"


In [4]:
df['Location'] = 'San Francisco'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,214 Putnam St,3 Beds,4 Baths,"2,255 sqft",1996,Attached Garage,No,"$998,000",San Francisco
1,1125 Vicente St,3 Beds,2 Baths,"2,018 sqft",1936,No Info,No,"$998,000",San Francisco
2,646 De Haro St,2 Beds,2 Baths,"1,515 sqft",1934,Attached Garage,No,"$1,395,000",San Francisco
3,89 Vienna St,2 Beds,1 Bath,,1900,Attached Garage,No,"$899,000",San Francisco
4,134-136 Hartford St,2 Beds,2 Baths,"2,000 sqft",1900,0 Open Spaces,,"$895,000",San Francisco
...,...,...,...,...,...,...,...,...,...
995,4 Jennifer Pl,1 Bed,1 Bath,"1,284 sqft (on 0.39 acres)",2000,Attached Garage,,"$998,000",San Francisco
996,Residence #215 Plan in Fulton 555 by Z&L Prope...,1 Bed,1 Bath,708 sqft,2021,No Info,,"$719,000+",San Francisco
997,2177 3rd St #505,1 Bed,1 Bath,719 sqft,2020,No Info,No,"$985,000",San Francisco
998,85 Columbia Square St,6 Beds,5 Baths,"4,998 sqft",1921,Garage,,"$3,500,000",San Francisco


In [5]:
df.to_csv('df_san_francisco.csv', index=False)